In [286]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime

def get_match_data(start_date,time_now_formatted,devengine):


    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    #Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay

# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today =time_now
two_years_ago = (today - datetime.timedelta(days=365*2)).strftime("%Y-%m-%d")

elo_hard, elo_clay, elo_data_hard, elo_data_clay = get_match_data(two_years_ago,time_now_formatted,devengine)

def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0

def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df

results_hard=get_filtered_data(elo_data_hard,elo_hard)
results_clay=get_filtered_data(elo_data_clay,elo_clay)
def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv('mensserving.csv')

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv('womensserving.csv')

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        #serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(result_df, serving, how='left', left_on=['Fav'], right_on=['Name'])
            result = pd.merge(result,serving, how='left', left_on=['Dog'], right_on=['Name'])
            result.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,)
            result.drop(columns=['Name_x','Name_y'],inplace=True)
    

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None
        
    return result, result_serving


_,serving_hard=process_serving_data(results_hard)
_,serving_clay=process_serving_data(results_clay)

elo_hardxx = pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)

data_concat = pd.DataFrame(columns=['Date', 'Player', 'Odds', 'Win/Loss'])
for dataset_type in [('Winner', 'Win'), ('Loser', 'Loss')]:
    df = elo_hard[['Date', dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df['Player'] = df[dataset_type[0]]
    df['Odds'] = df[f"{dataset_type[0]}_Odds"]
    df['Win/Loss'] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat['Odds'] = data_concat.Odds.astype(float)

def analyse_matchups(result_df,data_concat):
    for _,matchup in result_df.iterrows():
        player1=matchup.Fav
        player2=matchup.Dog
        player1_odds=float(matchup.Fav_Odds)
        player1_odds_hi=player1_odds+0.2
        player1_odds_lo=1
        player2_odds=float(matchup.Dog_Odds)
        player2_odds_hi=player2_odds+0.5
        player2_odds_lo=2
        player1=data_concat[data_concat['Player']==player1].copy()
        player2=data_concat[data_concat['Player']==player2].copy()
        player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
        if len(player2)>0:
            winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
        else:
            winperc2=0
        player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
        if len(player1)>0:
            winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
        else:
            winperc1=0
        if len(player1)>5 and len(player2)>3:
            print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
            print(len(player1),winperc1,len(player2),winperc2)

analyse_matchups(serving_hard,data_concat)
final_hard=serving_hard
final_clay=serving_clay

Error occured while merging the dataframes.
Error: 'Fav'
16:30 Andrew Paulson (1-->1.87) Sasi Kumar Mukund (2-->2.59)
9 0.7777777777777778 13 0.5384615384615384
16:30 Jason Jung (1-->1.5) Vladyslav Orlov (2-->3.73)
12 0.8333333333333334 6 0.16666666666666666
18:00 Madison Keys (1-->1.57) Jasmine Paolini (2-->3.56)
23 0.782608695652174 19 0.5789473684210527
18:00 Dayana Yastremska (1-->1.78) Alize Cornet (2-->2.86)
17 0.4117647058823529 13 0.23076923076923078
18:00 Barbora Krejcikova (1-->1.48) Irina Camelia Begu (2-->4.14)
21 0.8095238095238095 17 0.5294117647058824
19:30 Marie Bouzkova (1-->2.08) Sofia Kenin (2-->2.41)
44 0.7045454545454546 5 0.4
19:30 Yasutaka Uchiyama (1-->1.7) Ramkumar Ramanathan (2-->2.93)
11 0.8181818181818182 19 0.3684210526315789
19:30 Qinwen Zheng (1-->1.51) Shuai Zhang (2-->3.93)
18 0.7222222222222222 29 0.27586206896551724
19:30 Petra Kvitova (1-->1.3) Martina Trevisan (2-->7.41)
9 0.8888888888888888 18 0.2222222222222222
20:00 Fabian Marozsan (1-->1.72) Ale

In [287]:
elo_data_hard[['Time','Dog','Winner','Loser','Resulted','Winner_Odds','Dog_Rank','Fav_Rank']][(elo_data_hard['Dog_Rank']<elo_data_hard['Fav_Rank'])&(elo_data_hard['Dog_Rank']<100)].sort_values(by='Time')

,Time,Dog,Winner,Loser,Resulted,Winner_Odds,Dog_Rank,Fav_Rank
3,18:00,Alize Cornet,Dayana Yastremska,Alize Cornet,False,1.58,60.0,99.0
1,19:30,Shuai Zhang,Qinwen Zheng,Shuai Zhang,False,1.31,22.0,24.0
30,22:30,Nikoloz Basilashvili,Nikoloz Basilashvili,Liam Broady,False,1.99,84.0,140.0


In [288]:
final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)

In [289]:
final_hard[(final_hard['Fav_Serve%']>final_hard['Dog_Serve%'])&(final_hard['Dog_Return%']>10)&(final_hard['Fav_Serve%']>0)&(final_hard['Dog_Odds']<2.5)&(final_hard['Dog_Serve%']>70)]


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%


In [290]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Dayana Yastremska beat Alize Cornet
Alize Cornet beat Dayana Yastremska
Marius Copil beat Vitaliy Sachko
Mathias Bourgue beat Stuart Parker
Leylah Fernandez beat Julia Grabher
James McCabe beat Pratap Singh Digvijay
Alibek Kachmazov beat Akira Santillan
Yunseong Chung beat Dayne Kelly
Damir Dzumhur beat Otto Virtanen
Yasutaka Uchiyama beat Ramkumar Ramanathan


In [291]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

In [292]:
filter1=(final_hard['Sex']=='Womens') & \
(final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.9) & (final_hard['fav_percent']>0.1) & (final_hard['Dog_Odds']<2.5) & (((final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.1) & (final_hard['fav_percent']<0.5) & 
(final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.1) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.4) & (final_hard['fav_percent']>0.3)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | ((final_hard['dog_percent']>0.2) & (final_hard['dog_percent']<0.3) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.2) & (final_hard['dog_percent']<0.3) & (final_hard['fav_percent']<0.9) & (final_hard['fav_percent']>0.8)) | ((final_hard['dog_percent']>0.3) & (final_hard['dog_percent']<0.4) & (final_hard['fav_percent']<0.3) & (final_hard['fav_percent']>0.2)) | ((final_hard['dog_percent']>0.4) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)))
filter2=(final_hard['Sex']=='Womens') & \
           (((final_hard['dog_percent']>0.4) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.8) & (final_hard['fav_percent']>0.7)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.3) & (final_hard['fav_percent']>0.2)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
            ((final_hard['dog_percent']>0.7) & (final_hard['dog_percent']<0.8) & (final_hard['fav_percent']<0.4) & (final_hard['fav_percent']>0.3)) | 
            ((final_hard['dog_percent']>0.7) & (final_hard['dog_percent']<0.8) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6)) | 
            ((final_hard['dog_percent']>0.8) & (final_hard['dog_percent']<0.9) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | 
            ((final_hard['dog_percent']>0.8) & (final_hard['dog_percent']<0.9) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6)) | 
            ((final_hard['dog_percent']>0.9) & (final_hard['dog_percent']<1.0) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
            ((final_hard['dog_percent']>0.9) & (final_hard['dog_percent']<1.0) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6))) & \
           (final_hard['Dog_Odds']<2.5)
final_hard[filter1|filter2]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%


In [293]:
final_hard[
    (final_hard["Sex"] == "Mens")
    & (
        (
            (final_hard["dog_percent"] > 0.0)
            & (final_hard["dog_percent"] < 0.1)
            & (final_hard["fav_percent"] < 0.8)
            & (final_hard["fav_percent"] > 0.7)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.1)
            & (final_hard["dog_percent"] < 0.2)
            & (final_hard["fav_percent"] < 0.8)
            & (final_hard["fav_percent"] > 0.7)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.1)
            & (final_hard["dog_percent"] < 0.2)
            & (final_hard["fav_percent"] < 0.9)
            & (final_hard["fav_percent"] > 0.8)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.3)
            & (final_hard["dog_percent"] < 0.4)
            & (final_hard["fav_percent"] < 0.4)
            & (final_hard["fav_percent"] > 0.3)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.6)
            & (final_hard["dog_percent"] < 0.7)
            & (final_hard["fav_percent"] < 0.3)
            & (final_hard["fav_percent"] > 0.2)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.6)
            & (final_hard["dog_percent"] < 0.7)
            & (final_hard["fav_percent"] < 0.5)
            & (final_hard["fav_percent"] > 0.4)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.4)
            & (final_hard["fav_percent"] > 0.3)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.5)
            & (final_hard["fav_percent"] > 0.4)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.7)
            & (final_hard["fav_percent"] > 0.6)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.9)
            & (final_hard["dog_percent"] < 1.0)
            & (final_hard["fav_percent"] < 0.7)
            & (final_hard["fav_percent"] > 0.6)
            & (final_hard["Dog_Odds"] < 2.5)
        )
    )
]


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
29,22:30,1.77,1.99,Liam Broady,Liam Broady,32%,19,Nikoloz Basilashvili,31%,13,0.315789,0.307692,Mens,76.0,18.0,70.0,16.0


In [294]:
#100%
elo_data_hard['Fav_Odds']=elo_data_hard['Fav_Odds'].astype(float)
elo_data_hard[(elo_data_hard['Dog_Rank']>elo_data_hard['Fav_Rank'])&(elo_data_hard['Fav_Odds']<1.2)&(elo_data_hard['Fav_Odds']>1.1)&(elo_data_hard['Dog_Rank']>200)&(elo_data_hard['Fav_Rank']<50)]

,Surface,Date,Sex,Winner,Loser,Winner_Odds,Loser_Odds,Resulted,Time,Elo_Fav,...,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Rank,Elo_Dog_Rank,Fav,Dog,Fav_Odds,Dog_Odds,Fav_Rank,Dog_Rank
15,Hard,2023-02-19,Womens,Amanda Anisimova,Vera Zvonareva,1.17,4.98,False,21:00,Amanda Anisimova,...,1641.121997,1558.280767,35.0,435.0,Amanda Anisimova,Vera Zvonareva,1.17,4.98,35.0,435.0


In [295]:
final_hard#[final_hard['dog_percent']>0.5]#[(final_hard['Fav_Serve%']<final_hard['Dog_Serve%'])&(final_hard['Dog_Return%']>20)&(final_hard['Fav_Return%']<20)&(final_hard['Fav_Serve%']>0)&(final_hard['Dog_Odds']<2.5)]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
0,16:30,1.67,2.09,Andrew Paulson,Andrew Paulson,60%,10,Sasi Kumar Mukund,55%,11,0.600000,0.545455,Mens,0.0,0.0,NaN,NaN
1,16:30,1.30,3.23,Jason Jung,Jason Jung,50%,10,Vladyslav Orlov,40%,10,0.500000,0.400000,Mens,0.0,0.0,0.0,0.0
2,18:00,1.37,3.06,Madison Keys,Madison Keys,72%,32,Jasmine Paolini,41%,22,0.718750,0.409091,Womens,73.8,31.4,65.7,36.8
3,18:00,1.58,2.36,Dayana Yastremska,Dayana Yastremska,33%,30,Alize Cornet,46%,28,0.333333,0.464286,Womens,65.0,28.3,NaN,NaN
4,18:00,1.28,3.64,Barbora Krejcikova,Barbora Krejcikova,56%,34,Irina Camelia Begu,52%,23,0.558824,0.521739,Womens,74.3,35.0,NaN,NaN
5,18:00,1.16,4.55,Prajnesh Gunneswaran,Prajnesh Gunneswaran,55%,11,Benjamin Lock,42%,12,0.545455,0.416667,Mens,60.0,20.0,0.0,0.0
6,18:00,1.55,2.30,Hiroki Moriya,Hiroki Moriya,40%,10,Sidharth Rawat,17%,6,0.400000,0.166667,Mens,70.0,29.0,0.0,0.0
7,19:00,1.41,2.69,Evgeny Donskoy,Evgeny Donskoy,80%,10,Marko Topo,25%,8,0.800000,0.250000,Mens,0.0,0.0,61.0,0.0
8,19:30,1.88,1.91,Marie Bouzkova,Marie Bouzkova,100%,11,Sofia Kenin,27%,22,1.000000,0.272727,Womens,73.4,38.9,63.5,23.3
9,19:30,1.40,2.72,Giovanni Mpetshi Perricard,Giovanni Mpetshi Perricard,50%,10,Dominik Palan,30%,10,0.500000,0.300000,Mens,0.0,0.0,0.0,0.0


In [296]:
final_clay